In [28]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from aacCRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "securePassword123"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'grazioso_logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(
        html.A(
            href='https://www.snhu.edu',
            target='_blank', # _blank target opens a new tab for the link
            children=[
                html.Img(
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    # I scaled the image to 25% width and height because it was HUGE
                    style={'width': '25%', 'height': '25%'}
                )
            ]
        )
    ),
    html.Center(html.H3('Jonathan Thomas')),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        # Radio buttons for filtering results by animals that best fit
        # the specifications for each rescue type
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain Rescue', 'value': 'MR'},
                {'label': 'Disaster Rescue', 'value': 'DR'},
                {'label': 'Reset', 'value': 'RESET'}
            ],
            value=None # No radio button default selected
        )
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[
                             {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                         ],
                         data=df.to_dict('records'),
                         page_size=10,
                         sort_action='native',
                         style_table={'overflowX': 'auto'},
                         row_selectable='single',
                         selected_rows=[0],
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={'width': '50vw', 'height': '500px'}
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'),
              Output('datatable-id', 'columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'WR':
        # Water Rescue dog criteria:
        # -intact female
        # - breed: Labrador Retriever Mix, Chesa Bay Retr, Newfoundland
        # - age: 26wk - 156 wk
        # Because some valid candidates are mixed with other specific breeds
        # (see German Shepherd/Labrador Retriever), we can use regex
        # to match against all dogs with valid breeds in the breed column
        # We must ensure that the Labs are mix breeds though. No pure breds
        # are listed in the criteria
        df = pd.DataFrame(list(shelter.read(
            {'$and': # this indicates that ALL of the following criteria must be met
                [{'sex_upon_outcome': 'Intact Female'},
                 {'$or': [ # this indicates that ANY of the following criteria must be met
                     # this matches any breed that is formatted as XXXXLabrador RetrieverYYYY where XXXX is any non-zero collection of any non-newline characters and YYYY is any maybe-zero collection of non-newline characters
                     {'breed': {'$regex': '.+Labrador Retriever.*'}},
                     # this matches any non-zero collection of non-newline characters AFTER Labrador Retriever
                     {'breed': {'$regex': '.*Labrador Retriever.+'}},
                     # this matches against Chesa Bay Retr mixes with chars before/after
                     {'breed': {'$regex': '.+Chesa Bay Retr.*'}},
                     {'breed': {'$regex': '.*Chesa Bay Retr.+'}},
                     # and the same matching patterns for Newfoundland
                     # per the specs, Newfoundland pure breds are allowed
                     # (Mix not explicitly stated) so we can just use the one pattern
                     # to match ANY dog that has Newfoundland anywhere in the breed name
                     {'breed': {'$regex': '.*Newfoundland.*'}}
                     # turns out not a single Newfoundland breed in the dataset is
                     # within the valid age range.
                     # No Newfoundlands appearing on the pie chart is therefore, NOT a bug
                 ]},
                 {'$and': [
                     # matching dog must be 26 weeks or older
                     {'age_upon_outcome_in_weeks': {'$gte': 26}},
                     # AND must be 156 weeks or younger
                     {'age_upon_outcome_in_weeks': {'$lte': 156}}
                 ]}
                ]
            })))
        
    elif filter_type == 'MR':
        # Mountain Rescue Criteria:
        # - intact male
        # - breed: German Shepherd, Alaskan Malamute, Old English Sheepdog, 
        #          Siberian Husky, Rottweiler
        # - age: 26wk - 156wk
        # No mixes specified so we will only match against the exact breed names
        df = pd.DataFrame(list(shelter.read(
            {'$and':
                 [{'sex_upon_outcome': 'Intact Male'},
                  {'$or': [
                      {'breed': 'German Shepherd'},
                      {'breed': 'Alaskan Malamute'},
                      {'breed': 'Old English Sheepdog'},
                      {'breed': 'Siberian Husky'},
                      {'breed': 'Rottweiler'}
                  ]},
                  {'$and': [
                      {'age_upon_outcome_in_weeks': {'$gte': 26}},
                      {'age_upon_outcome_in_weeks': {'$lte': 156}}
                  ]}
                 ]
            })))
        
    elif filter_type == 'DR':
        # Disaster Rescue Criteria:
        # - intact male
        # - breed: Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound,
        #          Rottweiler
        # - age: 20wk - 300wk
        # Again, no mixes specified so we only match against exact breed names
        df = pd.DataFrame(list(shelter.read(
            {'$and':
                 [{'sex_upon_outcome': 'Intact Male'},
                  {'$or': [
                      {'breed': 'Doberman Pinscher'},
                      {'breed': 'German Shepherd'},
                      {'breed': 'Golden Retriever'},
                      {'breed': 'Bloodhound'},
                      {'breed': 'Rottweiler'}
                  ]},
                  {'$and': [
                      {'age_upon_outcome_in_weeks': {'$gte': 20}},
                      {'age_upon_outcome_in_weeks': {'$lte': 300}}
                  ]}
                 ]
            })))
        
    else:
        # If filter_type is RESET or anything else (maybe a glitch?)
        # return the normal, unfiltered results
        df = pd.DataFrame.from_records(shelter.read({}))
    
    
    df.drop(columns=['_id'],inplace=True) # drop _id column here to avoid errors with MongoDB v5+
    columns = [{'name': i, 'id': i} for i in df.columns]
    data = df.to_dict('records')
    
    return data, columns

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # put the currently displayed data in a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Error handling - return an empty div if 'breed' is not found in viewData columns
    # This should resolve itself as the page finishes loading and the callback keeps
    # executing
    if 'breed' not in dff.columns:
        return html.Div()
        
    # Get the breed names and counts of each
    breeds = dff['breed'].value_counts().keys().tolist()
    counts = dff['breed'].value_counts().tolist()
    # return a pie chart made from the above data
    return [
        dcc.Graph(
            figure = px.pie(
                data_frame = dff,
                values = counts,
                names = breeds
            )
        )
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is not None:
        return [{
            'if': { 'column_id': i },
            'background_color': '#D2F3FF'
        } for i in selected_columns]

app.config.suppress_callback_exceptions = True
# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        index = [0] # default select first row if none other selected
    else: 
        row = index[0]
        
    # Instead of centering on Austin city center, I want the map to center on the marker
    # and auto update whenever a new marker is used.
    # To do this, we grab the lat and long columns from the selected row
    # and use these to center instead of the Austin coordinates
    lat = dff.iloc[row, 13]
    long = dff.iloc[row, 14]
    
    return [
        # The map is styled so it only ever uses 50% of the view width for better appearances of thinner browser windows
        dl.Map(
            style={'width': '50vw', 'height': '500px'},
            center=[lat, long], 
            zoom=10, 
            children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                # Column 13 and 14 define the grid-coordinates for the map
                # Column 4 defines the breed for the animal
                # Column 9 defines the name of the animal
                dl.Marker(
                    position=[lat,long], 
                    children=[
                        dl.Tooltip(dff.iloc[row,4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row,9])
                        ])
                    ]
                )
            ]
        )
    ]

app.run_server(debug=False)

Dash app running on http://127.0.0.1:29805/
